In [1]:
import torch
from PIL import Image
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
from torch import optim
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
class SserafimDataset(Dataset):
    def __init__(self, path):
        self.path = path
        self.transform = transforms.Compose([
            transforms.Resize((480, 480)),
            transforms.ToTensor()
        ])
        self.images, self.labels, self.map = [], [], {}
        current_label = 0
        for label in os.listdir(path):
            label_path = os.path.join(path, label)
            if os.path.isdir(label_path):
                if label not in self.map:
                    self.map[label] = current_label
                    current_label += 1
                for img in os.listdir(label_path):
                    img_path = os.path.join(label_path, img)
                    self.images.append(img_path)
                    self.labels.append(self.map[label])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        image = self.transform(image)
        return image, label

In [3]:
data_path = os.path.join(os.getcwd(), 'data')
train_path = os.path.join(data_path, 'train')
test_path = os.path.join(data_path, 'test')
train_dataset = SserafimDataset(train_path)
test_dataset = SserafimDataset(test_path)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 8, 3, padding=1),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.MaxPool2d(2, 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(8, 16, 3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2, 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.5))
        self.fc1 = nn.Sequential(
            nn.Linear(32 * 60 * 60, 128),
            nn.ReLU())
        self.fc2 = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.5))
        self.fc3 = nn.Sequential(
            nn.Linear(64, 5))

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=0.1, patience=10)
criterion = nn.CrossEntropyLoss()
train_accs, test_accs = [], []
num_epochs = 50
max_acc = 0
for epoch in range(num_epochs):
    model.train()
    total_loss, correct_train, total_train = 0.0, 0, 0
    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        predicted = torch.argmax(output, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)
    train_acc = 100 * correct_train / total_train
    train_accs.append(train_acc)
    model.eval()
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for images, labels in test_dataloader:
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            predicted = torch.argmax(output, 1)
            correct_test += (predicted == labels).sum().item()
            total_test += labels.size(0)
    test_acc = 100 * correct_test / total_test
    test_accs.append(test_acc)
    scheduler.step(test_acc)
    if test_acc > max_acc:
        max_acc = test_acc
        torch.save(model.state_dict(), 'CNN4.pth')
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_dataloader):.4f}, Train Acc: {train_acc:.2f}%, Test Acc: {test_acc:.2f}%")
print("Done")


Epoch 1/50, Loss: 1.6217, Train Acc: 20.31%, Test Acc: 20.00%
Epoch 2/50, Loss: 1.6143, Train Acc: 19.10%, Test Acc: 20.80%
Epoch 3/50, Loss: 1.5811, Train Acc: 24.83%, Test Acc: 28.00%
Epoch 4/50, Loss: 1.5569, Train Acc: 30.03%, Test Acc: 24.80%
Epoch 5/50, Loss: 1.5074, Train Acc: 34.90%, Test Acc: 25.60%
Epoch 6/50, Loss: 1.4683, Train Acc: 40.28%, Test Acc: 30.40%
Epoch 7/50, Loss: 1.4169, Train Acc: 43.23%, Test Acc: 32.80%
Epoch 8/50, Loss: 1.3529, Train Acc: 46.35%, Test Acc: 34.40%
Epoch 9/50, Loss: 1.2989, Train Acc: 51.22%, Test Acc: 42.40%
Epoch 10/50, Loss: 1.2002, Train Acc: 56.25%, Test Acc: 32.00%
Epoch 11/50, Loss: 1.1353, Train Acc: 59.90%, Test Acc: 31.20%
Epoch 12/50, Loss: 1.0846, Train Acc: 60.24%, Test Acc: 38.40%
Epoch 13/50, Loss: 0.9869, Train Acc: 66.32%, Test Acc: 36.80%
Epoch 14/50, Loss: 0.9145, Train Acc: 72.22%, Test Acc: 34.40%
Epoch 15/50, Loss: 0.8551, Train Acc: 73.26%, Test Acc: 39.20%
Epoch 16/50, Loss: 0.7804, Train Acc: 76.56%, Test Acc: 36.80%
E